In [ ]:
!pip install -Uqq fastbook nbdev ipywidgets

In [ ]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

# from google.colab import drive
# drive.mount('/content/gdrive')
# BASE_DIR = '/content/gdrive/MyDrive/Colab Notebooks'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
def filesToStackedTensors(d1, d2):
  t = [tensor(Image.open(f)).to(device) for f in (path/d1/d2).ls().sorted()]
  return torch.stack(t).float()/255

def x_y(stacked_x, stacked_y):
  x = torch.cat([stacked_x, stacked_y]).view(-1, 28*28)
  y = tensor([1]*len(stacked_x) + [0]*len(stacked_y)).unsqueeze(1).to(device)
  return (x, y)

train_x, train_y = x_y(filesToStackedTensors('train', '3'), \
                       filesToStackedTensors('train', '7'))
valid_x, valid_y = x_y(filesToStackedTensors('valid', '3'), \
                       filesToStackedTensors('valid', '7'))

dset = list(zip(train_x, train_y))
valid_dset = list(zip(valid_x, valid_y))
dl = DataLoader(dset, batch_size=255, shuffle=True)
valid_dl = DataLoader(valid_dset, batch_size=255, shuffle=True)
dls = DataLoaders(dl, valid_dl)

In [ ]:
# model

def linear1(x, w, b): return x@w + b

# model parameters

def init_params(size): return (torch.randn(size)).to(device).requires_grad_()

# loss

def loss(preds, y):
  s = preds.sigmoid()
  return torch.where((y == 1), 1-s, s).float().mean()

# optimiser

class BasicOptimiser:
  def __init__(self, params, lr):
    self.params = params
    self.lr = lr

  def step(self):
    for p in self.params:
      p.data = p.data - p.grad.data * self.lr

  def zero_grad(self):
    for p in self.params:
      p.grad = None

# metric

def batch_accuracy(xb, yb):
  p = xb.sigmoid()
  return ((p > 0.5).float() == yb).float().mean()

def batch_accuracy_dl(dl, model, w, b):
  accs = [batch_accuracy(model(xb, w, b), yb) for xb, yb in dl]
  return torch.stack(accs).mean()

# fit

def calc_gradient(xb, yb, model, w, b):
    p = model(xb, w, b)
    l = loss(p, yb)
    l.backward()

def train_epoch(dl, model, opt, w, b):
  for xb, yb in dl:
    calc_gradient(xb, yb, model, w, b)
    opt.step()
    opt.zero_grad()

def train_model(dl, model, opt, n, w, b, pr=False):
  for i in range(n):
    train_epoch(dl, model, opt, w, b)
    if pr:
      print(f'acc after {i} epochs: {batch_accuracy_dl(dl, model, w, b)}')

# putting it all together

weights = init_params((28*28, 1))
bias = init_params(1)
lr = 0.1
opt = BasicOptimiser((weights, bias), lr)
train_model(dl, linear1, opt, 10, weights, bias, pr=True)

# learner

In [ ]:
# model

# def model(xs, ys):
#   def init_params(size): return (torch.randn(size)).to(device).requires_grad_()
#   w = init_params(xs)
#   b = init_params(ys)
#   def linear1(x): return x@w + b
#   return (linear1, (w, b))

class Linear:
  def init_params(size):
    g = torch.Generator()
    g.manual_seed(5)
    return (torch.randn(size, generator=g)).to(device).requires_grad_()
  def __init__(self, xsz, ysz):
    w = init_params(xsz)
    b = init_params(ysz)
    self.parameters = [w, b]
  def model(self):
    return lambda xb: xb@self.parameters[0] + self.parameters[1]
  def params(self):
    return self.parameters

def batch_accuracy_dl(dl, model):
  accs = [batch_accuracy(model(xb), yb) for xb, yb in dl]
  return torch.stack(accs).mean()

# fit

def calc_gradient(xb, yb, model):
    p = model(xb)
    l = loss(p, yb)
    print(f'loss: {l}')
    l.backward()

def train_epoch(dl, model, opt):
  for xb, yb in dl:
    p = model(xb)
    l = loss(p, yb)
    # print(f'loss: {l}')
    l.backward()
    opt.step()
    opt.zero_grad()

def train_model(dl, model, opt, n, pr=False):
  for i in range(n):
    train_epoch(dl, model, opt)
    if pr:
      print(f'acc after {i} epochs: {batch_accuracy_dl(dl, model)}')

# putting it all together

lr = 1.
linear1 = Linear((28*28, 1), 1)
params = linear1.params()
opt = BasicOptimiser(params, lr)
train_model(dl, linear1.model(), opt, 10, pr=True)

In [ ]:
for x in (1,2,3,4):
  print(x)